## Retrieval augmented generation
 
In retrieval augmented generation (RAG), an LLM retrieves contextual documents from an external dataset as part of its execution. 

This is useful if we want to ask question about specific documents (e.g., our PDFs, a set of videos, etc). 

In [ ]:
%pip install langchain openai dotenv

##### 小筆記：為什麼用 %pip 會比 !pip 還好？
解答：
* !pip = 在旁邊開個小視窗偷偷叫系統去幫你安裝，結果可能裝到別的房間。
* %pip = 直接跟 Notebook 的 kernel 說「幫我裝這個套件」，保證裝到正確的房間。

In [23]:
'''
目的：建立langchain + openai 的基礎環境
'''
#! pip install langchain
import os # 作業系統相關功能（讀取環境變數）
import openai # openai api 客戶端
# import sys # python 系統功能，下載到地端用不到
# sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv # dotenv 是專門用來讀取.env套件的套件，並接上環境
_ = load_dotenv(find_dotenv()) # 讀取.env檔案

'''
為什麼要這樣寫？
find_dotenv() → 自動尋找 .env 檔案（向上搜尋資料夾）
load_dotenv() → 載入 .env 檔案到環境變數
_ = → 把回傳值丟掉（不需要）
load_dotenv() 會回傳什麼？
ans:會回傳true, false，但我們只要結果就好不需要回傳值
一般 : "result = load_doenv()" → 有回傳值
如果不要 : "_ = load_doenv()" → 自動忽略回傳值（pythin 慣例）

'''

# 載入 api key
openai.api_key  = os.environ['OPENAI_API_KEY']

## PDFs

Let's load a PDF [transcript](https://see.stanford.edu/materials/aimlcs229/transcripts/MachineLearning-Lecture01.pdf) from Andrew Ng's famous CS229 course! These documents are the result of automated transcription so words and sentences are sometimes split unexpectedly.

In [ ]:
%pip install -U langchain-community pypdf
'''
套件介紹
1. langchain-community：包含現在要使用的document_loaders、向量資料庫、llm整合（hugging face, anthropic）
2. pypdf：專門處理pdf檔案得python套件（用來解析pdf）
-U 是什麼？
ans：等於--upgrade，如果已經裝了，升級到最新的版本
'''

In [26]:
# The course will show the pip installs you would need to install packages on your own machine.
# These packages are already installed on this platform and should not be run again.
#! pip install pypdf 

from langchain.document_loaders import PyPDFLoader # PyPDFLoader：LangChain 的 PDF 文件載入器，專門把pdf檔案轉換成langchain形式
# 建立載入器實例（指定來源）
loader = PyPDFLoader("https://see.stanford.edu/materials/aimlcs229/transcripts/MachineLearning-Lecture01.pdf")
# 實際載入檔案：下載/讀取 PDF → 用 pypdf 解析 → 分頁處理 → 轉成 Document 格式
pages = loader.load()

In [28]:
# 實際看看結果
print(f"總共載入了 {len(pages)} 頁")
print(f"第一頁內容：{pages[0].page_content[:200]}...")
print(f"第一頁 metadata：{pages[0].metadata}")



# len(pages) → 了解有多少頁
# 字串切片 + .page_content → 看到某頁的頁面內容
# 字串切片 + .metadata → 看到某頁的 meta data

總共載入了 22 頁
第一頁內容：MachineLearning-Lecture01  
Instructor (Andrew Ng): Okay. Good morning. Welcome to CS229, the machine 
learning class. So what I wanna do today is just spend a little time going over the logistics 
of...
第一頁 metadata：{'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2008-07-11T11:25:23-07:00', 'author': '', 'moddate': '2008-07-11T11:25:23-07:00', 'title': '', 'source': 'https://see.stanford.edu/materials/aimlcs229/transcripts/MachineLearning-Lecture01.pdf', 'total_pages': 22, 'page': 0, 'page_label': '1'}


#### 小筆記
##### 1. meta data 的功能是什麼？
* 追蹤來源：如果是rag，有了mata data如果使用者詢問資料出處可以查詢。
* 過濾與搜尋：搜尋時可以只搜尋某一頁、某個日期、某個來源
    ```python
    # 只搜尋特定頁面
    first_5_pages = [p for p in pages if p.metadata['page'] < 5]
    ```
##### 2. 通常 meta data 有那些格式？
```python
{
    'source': 'https://see.stanford.edu/.../MachineLearning-Lecture01.pdf',
    'page': 0,           # 第幾頁（從0開始）
    'total_pages': 20,   # 總頁數
    'title': 'Machine Learning CS229 Lecture 1',
    'author': 'Stanford University',
    'creation_date': '2023-01-15',
    'file_size': '2.5MB'
}
```
##### 3. 為什麼程式裡沒有看到 pypdf 的作用，但還要安裝？
* LangChain 比較像是「封裝」或是中間轉換器，不會參與解析 pdf
    * 呼叫外部的 PDF 解析套件（如 pypdf）→ 把頁面內容抽出來
    * 再把抽出來的文字包裝成 LangChain 的 Document 物件，方便後面使用
* 真正解析的仍舊是 pypdf
* 這樣設計的原因：保持 LangChain 輕量，採取「lazy dependency（延遲依賴）」策略
* 備註：pdf 轉成langchain document 完全沒有用到 llm api～

## YouTube

#### 小筆記：音檔轉文字作法上的差異
* 課本示範方式：youtube影片 → 下載音檔  → llm轉換成文字 → 載入到langchain
    * 遇到的問題：是透過地端操作且用自己的openai api服務，所以此方法除了要下載許多套件、也要計費
    * 解決方法：將影片替換成更短的影片練習
* 偷吃步方式：直接讀取影片的逐字稿 → 載入到langchain（但這樣就沒有用ai)


In [ ]:
%pip install yt_dlp pydub ffmpeg-python ffmpeg ffprobe

In [ ]:
from langchain.document_loaders.generic import GenericLoader
from langchain_community.document_loaders import FileSystemBlobLoader # 2025.08.19 位置改了, 原本在document_loaders.generic裡面
from langchain.document_loaders.parsers import OpenAIWhisperParser 
from langchain.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader
import os 

#### 小筆記 - 各種套件介紹

音檔處理相關：

* yt_dlp：下載 youtube 音檔的工具
* pydub：python 音檔處理套件
* ffmpeg-python：讓 py 可以呼叫 ffmpeg 套件
* ffmpeg：Python 環境中的 ffmpeg 介面（注意：真正的 ffmpe 要去 homebrew 下載）

LangChain 載入器：

* GenericLoader： 通用載入＆解析框架
* FileSystemBlobLoader：從本地檔案系統載入檔案（載入器）
* OpenAIWhisperParser：把音檔轉為文字（解析器）
* YoutubeAudioLoader：youtube檔案載入（載入器）

運作流程：
1. YoutubeAudioLoader （和 FileSystemBlobLoader 擇一）
   
   ↓ (使用 yt_dlp)
   下載 YouTube 影片
   
2. ffmpeg/pydub
   
   ↓ (處理音檔格式)
   轉換成 Whisper 可用格式
   
3. FileSystemBlobLoader （和 YoutubeAudioLoader 擇一）
   
   ↓ (讀取本地檔案)
   載入處理好的音檔
   
4. OpenAIWhisperParser
   
   ↓ (呼叫 OpenAI API)
   轉錄成文字
   
5. GenericLoader
   
   ↓ (協調整個流程)
   包裝成 LangChain Document

In [3]:
url="https://www.youtube.com/watch?v=rEDzUT3ymw4"
save_dir="/Users/mangtinglee/Desktop/2025_gap_careerpath/ai_agent/youtube/"
os.makedirs(save_dir, exist_ok=True) # 這行會自動建立資料夾（如果不存在）
loader = GenericLoader( # GenericLoader 只能接受兩個參數
    # YoutubeAudioLoader([url],save_dir),  # fetch from youtube 下載到本地 － 讀取器之一
    FileSystemBlobLoader(save_dir, glob="*.mp4"),   #fetch locally － 讀取器之二
    OpenAIWhisperParser() # 透過 openai api 轉錄 - 也就是解析器
)
docs = loader.load()

Transcribing part 1!


'Transcribing part 1!' →  OpenAI Whisper API 的內建訊息

In [6]:
print(docs[0].metadata)


{'source': '/Users/mangtinglee/Desktop/2025_gap_careerpath/ai_agent/youtube/Explained In A Minute： Neural Networks.mp4', 'chunk': 0}


##### 小筆記 - GenericLoader 設計與功用
* GenericLoader ＝ 不同檔案的通用框架，也可以像pdf、urls一樣使用專用的框架也行（PyPDFLoader, WebBaseLoader）
* GenericLoader 的設計（只會接受2個參數）：
    ```python
    GenericLoader(
        blob_loader, # 負責「載入資料」
        parser # 負責「解析資料」
    )
    ``

## URLs

In [8]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://github.com/basecamp/handbook/blob/master/titles-for-programmers.md")

USER_AGENT environment variable not set, consider setting it to identify your requests.
